In [1]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 47.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 32.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 87.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.9/275.9 KB 90.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.1/345.1 KB 83.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 68.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 KB 54.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 KB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 KB 55.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 KB 73.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.3/130.3 KB 77.7 MB/s eta 0:00:00
  Using cached uvicorn-0.34.0-py3-none-any.whl (62 kB)
     ━━━━━━━━━

In [3]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [4]:
!PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

In [6]:
!ollama pull gemma3:12b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest 
pulling manifest      ▏    0 B/8.1 GB                  
pulling manifest      ▏    0 B/8.1 GB                  
pulling manifest      ▏    0 B/8.1 GB                  
pulling manifest      ▏ 4.3 MB/8.1 GB                  
pulling manifest      ▏  14 MB/8.1 GB                  
pulling manifest      ▏  21 MB/8.1 GB                  
pulling manifest      ▏  37 MB/8.1 GB                  
pulling manifest      ▏  45 MB/8.1 GB                  
pulling manifest      ▏  45 MB/8.1 GB                  
pulling manifest      ▏  56 MB/8.1 GB                  
pulling manifest      ▏  64 MB/8.1 GB   64 MB/s    2m5s
pulling manifest      ▏  69 MB/8.1 GB   64 MB/s    2m5s
pulling manifest      ▏  87 MB/8.1 GB   64 MB/s    2m5s
pulling manifest      ▏ 100 MB/8.1 GB   64 MB/s    2m5s
pulling manifest      ▏ 107 MB/8.1 GB   64 MB/s    2m5s
pulling manifest      ▏ 124 

In [7]:
!ollama pull gemma3:27b

pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠴ pulling manifest ⠦ pulling manifest 
pulling manifest      ▏    0 B/ 17 GB                  
pulling manifest      ▏    0 B/ 17 GB                  
pulling manifest      ▏ 1.5 MB/ 17 GB                  
pulling manifest      ▏ 6.1 MB/ 17 GB                  
pulling manifest      ▏  17 MB/ 17 GB                  
pulling manifest      ▏  30 MB/ 17 GB                  
pulling manifest      ▏  46 MB/ 17 GB                  
pulling manifest      ▏  53 MB/ 17 GB                  
pulling manifest      ▏  67 MB/ 17 GB                  
pulling manifest      ▏  81 MB/ 17 GB                  
pulling manifest      ▏  88 MB/ 17 GB   88 MB/s   3m15s
pulling manifest      ▏ 100 MB/ 17 GB   88 MB/s   3m15s
pulling manifest      ▏ 113 MB/ 17 GB   88 MB/s   3m15s
pulling manifest      ▏ 120 MB/ 17 GB   88 MB/s   3m15s
pulling manifest      ▏ 135 MB/ 17 GB   88 MB/s   3m14s
pulling manifest      ▏ 148 

In [10]:
import os
import re
import numpy as np
import pandas as pd
import gradio as gr
import torch
import faiss
import requests
import torch.nn.functional as F

from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModelForSequenceClassification
from sentence_transformers import SentenceTransformer

# Load models

tokenizer = AutoTokenizer.from_pretrained("phmtung/dpl302m-model")
query_decomposition = AutoModelForCausalLM.from_pretrained("phmtung/dpl302m-model", device_map='cuda')
query_decomposition.eval()

embedder = SentenceTransformer('dangvantuan/vietnamese-document-embedding', trust_remote_code=True)

classifi_model_path = "./phobert_finetuned"
classifi_model = AutoModelForSequenceClassification.from_pretrained(classifi_model_path, device_map='cuda')
classifi_tokenizer = AutoTokenizer.from_pretrained(classifi_model_path)
classifi_model.eval()

# Load faiss index and datastore
index = faiss.read_index('faiss.index')
df = pd.read_csv("chunks.csv")
df_folder = 'data_with_label'
faiss_index_folder = 'faiss_index'

dfs = sorted([f for f in os.listdir(df_folder) if not f.startswith('.')])
index_files = sorted([f for f in os.listdir(faiss_index_folder) if not f.startswith('.')])

for i in range(1, 6):
    globals()[f"df{i}"] = pd.read_csv(f"{df_folder}/{dfs[i-1]}")
    globals()[f"index{i}"] = faiss.read_index(f"{faiss_index_folder}/{index_files[i-1]}")

# Chat history storage
chat_history = []

###  HELPER FUNCTIONS  ###

def predict_label(query):
    """Predict law label of a given query
    Args:
        text (str): input query
    Returns:
        int: predicted label
    """
    inputs = classifi_tokenizer(query, return_tensors="pt", truncation=True, padding="max_length", max_length=128).to('cuda')
    
    with torch.no_grad():
        outputs = classifi_model(**inputs)
        
    logits = outputs.logits
    probs = F.softmax(logits, dim=-1)
    
    predicted_label = torch.argmax(probs, dim=-1).item()

    label_mapping = {0: "Trẻ em", 1: "Bình đẳng giới", 2: "Dân số", 3: "Hôn nhân và gia đình", 4: "Phòng, chống bạo lực gia đình", 5: "Khác"}

    return predicted_label+1

def get_embedding(query):
    """Get embedding of a given query
    Args:
        query (str): input query
    Returns:
        np.array: query embedding
    """
    return embedder.encode(query)


def similarity_search_with_filter(query, label, top_k=3):
    """Search for similar articles given a query and label
    Args:
        query (str): input query
        label (int): predicted label
        top_k (int): number of articles to return
    Returns:
        list[str]: list of articles
    """
    query_embedding = np.array(get_embedding(query)).reshape(1, -1)
    _, indices = globals()[f"index{label}"].search(query_embedding, top_k)

    results = []
    for i in indices[0]:
        row = globals()[f"df{label}"].iloc[i]
        raw_prefix = [row["Law"], row["Chapter"], row["Section"]]
        prefix = " ".join(filter(pd.notna, raw_prefix)) 
        
        article = f"{prefix}\n{row['Article']}"
        results.append(article)

    return results

def similarity_search(query, top_k=3):
    """Search for similar articles given a query
    Args:
        query (str): input query
        top_k (int): number of articles to return
    Returns:
        list[str]: list of articles
    """
    query_embedding = np.array(get_embedding(query)).reshape(1, -1)
    _, indices = index.search(query_embedding, top_k)

    results = []
    for i in indices[0]:
        row = df.iloc[i]
        raw_prefix = [row["Law"], row["Chapter"], row["Section"]]
        prefix = " ".join(filter(pd.notna, raw_prefix)) 

        article = f"{prefix}\n{row['Article']}" if prefix else row["Article"]
        results.append(article)

    return results

def decompose_question(query, query_decomposition, tokenizer):
    """Break down a query to 3 simpler sub-queries
    Args:
        query (str): input query
        query_decomposition (Llama 3.2 Instruct fine-tuned): query decomposition model
        tokenizer: Llama 3.2 Instruct's tokenizer
    Returns:
        list[str]: list of sub-queries
    """
    system_prompt = f"""You are a helpful assistant that generates multiple sub-questions related to an input question.
    The goal is to break down the input into a set of 3 sub-problems / sub-questions that can be answered in isolation and must be understandable stand-alone questions.
    Generate multiple search queries related to: {query}
    
    **Respond format:**
    Write a short analysis of why the query needs decomposition and how it will be done to derive the final 3 decomposed sub-questions.
    [Sub-question 1, Sub-question 2, Sub-question 3]"""

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": query},
    ]

    with torch.no_grad():
        inputs = tokenizer.apply_chat_template(messages, tokenize=True, return_tensors="pt").to("cuda")
        output_ids = query_decomposition.generate(input_ids=inputs, max_new_tokens=512, use_cache=True)
        
        decoded_output = tokenizer.decode(output_ids[0], skip_special_tokens=True)
        raw = re.search(r'\[([^\]]*)\](?!.*\[)', decoded_output, re.DOTALL).group(1)
        questions = [item.strip().strip("'") for item in raw.split(",")]
    #     del inputs
    #     del output_ids
    #     del decoded_output
    #     del raw
    

    # query_decomposition.to('cpu')
    # torch.cuda.empty_cache()
    # gc.collect()
        
    return questions


def query_ollama(payload):
    """Calls Ollama API to generate responses
    Args:
        payload (dict): {
            "model": id of model used to generate response,
            "messages": user's chat history,
            "stream": boolean value to determine if the chat is streamed,
            "keep_alive": how long the model will stay loaded into memory,
            "options": {
                "num_ctx": context length,
                "temperature": The temperature of the model,
                "top_k": Reduces the probability of generating nonsense,
                "top_p": Works together with top-k,
                }
            }
    Returns:
        chat_history (list[dict]): User's chat history
    """
    api_url = "http://localhost:11434/api/chat"  
    response = requests.post(api_url, json=payload)
    if response.status_code == 200:
        return response.json()["message"]["content"]
    else:
        return "Xin lỗi, có lỗi xảy ra khi gọi API."


def chatbot(query):
    global chat_history

    if not chat_history:  
        decompose_question_result = decompose_question(query, query_decomposition, tokenizer)
        # labels = [predict_label(q) for q in decompose_question_result]
        label = predict_label(query)

        # data1, data2, data3 = [similarity_search(decompose_question_result[i], labels[i]) if labels[i] != 6 else '' for i in range(len(decompose_question_result))]
        data1 = similarity_search_with_filter(query, label) if label != 6 else ''
        data2 = similarity_search(query)
        data3 = [similarity_search(query) for query in decompose_question_result]
        context = set(data1) | set(data2)
        for chunks in data3:
            for chunk in chunks:
                context.update(chunk)
        
        context_text = "\n".join(context)

        system_prompt = f"""Bạn là OpenAPI, một chatbot hỗ trợ pháp lý tại Việt Nam.
        Nhiệm vụ của bạn là trả lời một câu hỏi về pháp luật dựa trên ngữ cảnh sau đây.

        **Ngữ cảnh**: {context_text}

        **Vui lòng tuân thủ các yêu cầu sau**:
            1. **Luôn trả lời bằng tiếng Việt.** 
            2. **Trả lời ngắn gọn không lan man DƯỚI 100 TỪ và trình bày rõ ràng, dễ hiểu.**    
            3. **Chỉ sử dụng thông tin từ ngữ cảnh được cung cấp ở trên.** Tuyệt đối **không bịa đặt** thông tin. Nếu không thể trả lời dựa trên ngữ cảnh, hãy nói: "Xin lỗi, tôi không thể trả lời câu hỏi này của bạn."
            4. **Câu trả lời được dùng từ phần nào của ngữ cảnh thì phải trích dẫn nguồn**, quy tắc trích dẫn là nguồn bao gồm tên văn bản pháp luật (ví dụ như luật, nghị định, thông tư,...), chương, mục và điều khoản.  
                Ví dụ trích dẫn nguồn:  
                    - *Luật Hôn nhân và Gia đình, Chương II, Điều 8.*  
                    - *Bộ luật Lao động, Mục 3, Điều 34.*"""
            
        chat_history = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": query + " RESPONSE UNDER 150 TOKENS"}
        ]

        payload = {
            "model": 'gemma3:27b',
            "messages": chat_history,
            "stream": False,
            "keep_alive": 0,
            "options": {
                "num_ctx": 2048,
                "temperature": 0.0,
                "top_k": 64,
                "top_p": 0.9,
                }
            }
        
        response = query_ollama(payload=payload)
    else:
        system_prompt = f"""Bạn là OpenAPI, một chatbot hỗ trợ pháp lý tại Việt Nam.
        Bạn được được cung cấp một ngữ cảnh từ lịch sử cuộc trò chuyện với người dùng. Lịch sử trò chuyện này bao gồm các câu hỏi của người dùng, các Điều luật, bộ luật, thông tư etc. mà người dùng đã cung cấp kèm theo câu trả lời của chatbot cho mỗi câu hỏi.
        Nhiệm vụ của bạn là trả lời câu hỏi tiếp theo mà người dùng hỏi trong cuộc trò chuyện đó.

        **Vui lòng tuân thủ các yêu cầu sau**:
            1. **Luôn trả lời bằng tiếng Việt.**
            2. **Trả lời ngắn gọn không lan man DƯỚI 100 TỪ và trình bày rõ ràng, dễ hiểu.**     
            3. **Chỉ sử dụng thông tin từ ngữ cảnh được cung cấp. ** Tuyệt đối **không bịa đặt** thông tin. Nếu không thể trả lời dựa trên ngữ cảnh, hãy nói: "Xin lỗi, tôi không thể trả lời câu hỏi này của bạn."
            4. **Câu trả lời được dùng từ phần nào của ngữ cảnh thì phải trích dẫn nguồn**, quy tắc trích dẫn là nguồn bao gồm tên văn bản pháp luật (ví dụ như luật, nghị định, thông tư,...), chương, mục và điều khoản.  
                Ví dụ trích dẫn nguồn:  
                    - *Luật Hôn nhân và Gia đình, Chương II, Điều 8.*  
                    - *Bộ luật Lao động, Mục 3, Điều 34.*"""
        
        chat_history.extend([
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": query + " RESPONSE UNDER 150 TOKENS"}
        ])  

        payload = {
            "model": 'gemma3:27b',
            "messages": chat_history,
            "stream": False,
            "keep_alive": 0,
            "options": {
                "num_ctx": 3072,
                "temperature": 0.2,
                "top_k": 64,
                "top_p": 0.9,
                }
            }
        
        response = query_ollama(payload=payload)

    chat_history.append({"role": "assistant", "content": response})  

    return chat_history

def reset_chat():
    global chat_history
    # query_decomposition.to('cuda')
    chat_history = []
    return []

with gr.Blocks(theme="soft") as demo:
    gr.Markdown("<h2 style='text-align: center;'>Trợ lý Pháp luật Việt Nam 🇻🇳</h2>", elem_id="title")

    with gr.Column():
        chatbot_ui = gr.Chatbot(label="Trợ lý Pháp luật", type="messages", elem_id="chatbot", height=700)

    with gr.Row(elem_id="bottom_section"):
        query_input = gr.Textbox(placeholder="Nhập câu hỏi của bạn...", label="Câu hỏi", elem_id="input_box")

    with gr.Row():
        submit_btn = gr.Button(" ↗️ Gửi", elem_id="send_btn")
        reset_btn = gr.Button("  🔄 Đoạn chat mới", elem_id="reset_btn")

    submit_btn.click(chatbot, inputs=query_input, outputs=chatbot_ui)
    reset_btn.click(reset_chat, outputs=chatbot_ui)

    gr.Markdown("<p style='text-align: center; font-size: 12px; color: gray;'>🚀 Được phát triển bởi nhóm OPENAPI để hỗ trợ pháp luật</p>")

demo.launch(share=True)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://0ddb6124093fd9b237.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


### Evaluate multiple choices

In [9]:
import os
import re
import numpy as np
import pandas as pd
import gradio as gr
import torch
import faiss
import requests
import torch.nn.functional as F

from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModelForSequenceClassification
from sentence_transformers import SentenceTransformer

# Load models

tokenizer = AutoTokenizer.from_pretrained("phmtung/dpl302m-model")
query_decomposition = AutoModelForCausalLM.from_pretrained("phmtung/dpl302m-model", device_map='cuda')
query_decomposition.eval()

embedder = SentenceTransformer('dangvantuan/vietnamese-document-embedding', trust_remote_code=True)

classifi_model_path = "./phobert_finetuned"
classifi_model = AutoModelForSequenceClassification.from_pretrained(classifi_model_path, device_map='cuda')
classifi_tokenizer = AutoTokenizer.from_pretrained(classifi_model_path)
classifi_model.eval()

# Load faiss index and datastore
index = faiss.read_index('faiss.index')
df = pd.read_csv("chunks.csv")
df_folder = 'data_with_label'
faiss_index_folder = 'faiss_index'

dfs = sorted([f for f in os.listdir(df_folder) if not f.startswith('.')])
index_files = sorted([f for f in os.listdir(faiss_index_folder) if not f.startswith('.')])

for i in range(1, 6):
    globals()[f"df{i}"] = pd.read_csv(f"{df_folder}/{dfs[i-1]}")
    globals()[f"index{i}"] = faiss.read_index(f"{faiss_index_folder}/{index_files[i-1]}")

# Chat history storage
chat_history = []

###  HELPER FUNCTIONS  ###

def predict_label(query):
    """Predict law label of a given query
    Args:
        text (str): input query
    Returns:
        int: predicted label
    """
    inputs = classifi_tokenizer(query, return_tensors="pt", truncation=True, padding="max_length", max_length=128).to('cuda')
    
    with torch.no_grad():
        outputs = classifi_model(**inputs)
        
    logits = outputs.logits
    probs = F.softmax(logits, dim=-1)
    
    predicted_label = torch.argmax(probs, dim=-1).item()

    label_mapping = {0: "Trẻ em", 1: "Bình đẳng giới", 2: "Dân số", 3: "Hôn nhân và gia đình", 4: "Phòng, chống bạo lực gia đình", 5: "Khác"}

    return predicted_label+1

def get_embedding(query):
    """Get embedding of a given query
    Args:
        query (str): input query
    Returns:
        np.array: query embedding
    """
    return embedder.encode(query)


def similarity_search_with_filter(query, label, top_k=3):
    """Search for similar articles given a query and label
    Args:
        query (str): input query
        label (int): predicted label
        top_k (int): number of articles to return
    Returns:
        list[str]: list of articles
    """
    query_embedding = np.array(get_embedding(query)).reshape(1, -1)
    _, indices = globals()[f"index{label}"].search(query_embedding, top_k)

    results = []
    for i in indices[0]:
        row = globals()[f"df{label}"].iloc[i]
        raw_prefix = [row["Law"], row["Chapter"], row["Section"]]
        prefix = " ".join(filter(pd.notna, raw_prefix)) 
        
        article = f"{prefix}\n{row['Article']}"
        results.append(article)

    return results

def similarity_search(query, top_k=3):
    """Search for similar articles given a query
    Args:
        query (str): input query
        top_k (int): number of articles to return
    Returns:
        list[str]: list of articles
    """
    query_embedding = np.array(get_embedding(query)).reshape(1, -1)
    _, indices = index.search(query_embedding, top_k)

    results = []
    for i in indices[0]:
        row = df.iloc[i]
        raw_prefix = [row["Law"], row["Chapter"], row["Section"]]
        prefix = " ".join(filter(pd.notna, raw_prefix)) 

        article = f"{prefix}\n{row['Article']}" if prefix else row["Article"]
        results.append(article)

    return results

def decompose_question(query, query_decomposition, tokenizer):
    """Break down a query to 3 simpler sub-queries
    Args:
        query (str): input query
        query_decomposition (Llama 3.2 Instruct fine-tuned): query decomposition model
        tokenizer: Llama 3.2 Instruct's tokenizer
    Returns:
        list[str]: list of sub-queries
    """
    system_prompt = f"""You are a helpful assistant that generates multiple sub-questions related to an input question.
    The goal is to break down the input into a set of 3 sub-problems / sub-questions that can be answered in isolation and must be understandable stand-alone questions.
    Generate multiple search queries related to: {query}
    
    **Respond format:**
    Write a short analysis of why the query needs decomposition and how it will be done to derive the final 3 decomposed sub-questions.
    [Sub-question 1, Sub-question 2, Sub-question 3]"""

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": query},
    ]

    with torch.no_grad():
        inputs = tokenizer.apply_chat_template(messages, tokenize=True, return_tensors="pt").to("cuda")
        output_ids = query_decomposition.generate(input_ids=inputs, max_new_tokens=512, use_cache=True)
        
        decoded_output = tokenizer.decode(output_ids[0], skip_special_tokens=True)
        raw = re.search(r'\[([^\]]*)\](?!.*\[)', decoded_output, re.DOTALL).group(1)
        questions = [item.strip().strip("'") for item in raw.split(",")]
    #     del inputs
    #     del output_ids
    #     del decoded_output
    #     del raw
    

    # query_decomposition.to('cpu')
    # torch.cuda.empty_cache()
    # gc.collect()
        
    return questions


def query_ollama(payload):
    """Calls Ollama API to generate responses
    Args:
        payload (dict): {
            "model": id of model used to generate response,
            "messages": user's chat history,
            "stream": boolean value to determine if the chat is streamed,
            "keep_alive": how long the model will stay loaded into memory,
            "options": {
                "num_ctx": context length,
                "temperature": The temperature of the model,
                "top_k": Reduces the probability of generating nonsense,
                "top_p": Works together with top-k,
                }
            }
    Returns:
        chat_history (list[dict]): User's chat history
    """
    api_url = "http://localhost:11434/api/chat"  
    response = requests.post(api_url, json=payload)
    if response.status_code == 200:
        return response.json()["message"]["content"]
    else:
        return "Xin lỗi, có lỗi xảy ra khi gọi API."


def chatbot(query):
    global chat_history

    if not chat_history:  
        decompose_question_result = decompose_question(query, query_decomposition, tokenizer)
        # labels = [predict_label(q) for q in decompose_question_result]
        label = predict_label(query)

        # data1, data2, data3 = [similarity_search(decompose_question_result[i], labels[i]) if labels[i] != 6 else '' for i in range(len(decompose_question_result))]
        data1 = similarity_search_with_filter(query, label) if label != 6 else ''
        data2 = similarity_search(query)
        data3 = [similarity_search(query) for query in decompose_question_result]
        context = set(data1) | set(data2)
        for chunks in data3:
            for chunk in chunks:
                context.update(chunk)
        
        context_text = "\n".join(context)

        system_prompt = f"""Bạn là OpenAPI, một chatbot hỗ trợ pháp lý tại Việt Nam.
        Nhiệm vụ của bạn là trả lời các câu hỏi chắc nhiệm gồm các đáp án A, B, C và D về các văn bản quy phạm pháp luật dựa trên ngữ cảnh sau đây và kết hợp với lịch sử trò chuyện.
        Ngữ cảnh: {context_text}
        Vui lòng tuân thủ yêu cầu sau:
            1. **Chỉ trả lời A, B, C hoặc D. Không đề cập thêm bất cứ thông tin gì**"""
            
        chat_history = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": query }
        ]

        payload = {
            "model": 'gemma3:27b',
            "messages": chat_history,
            "stream": False,
            "keep_alive": 0,
            "options": {
                "num_ctx": 2048,
                "temperature": 0.0,
                "top_k": 64,
                "top_p": 0.9,
                }
            }
        
        response = query_ollama(payload=payload)
    else:
        system_prompt = f"""Bạn là OpenAPI, một chatbot hỗ trợ pháp lý tại Việt Nam.
        Bạn được được cung cấp một ngữ cảnh từ lịch sử cuộc trò chuyện với người dùng. Lịch sử trò chuyện này bao gồm các câu hỏi của người dùng, các Điều luật, bộ luật, thông tư etc. mà người dùng đã cung cấp kèm theo câu trả lời của chatbot cho mỗi câu hỏi.
        Nhiệm vụ của bạn là trả lời câu hỏi tiếp theo mà người dùng hỏi trong cuộc trò chuyện đó.

        **Vui lòng tuân thủ các yêu cầu sau**:
            1. **Luôn trả lời bằng tiếng Việt.**
            2. **Trả lời ngắn gọn không lan man DƯỚI 100 TỪ và trình bày rõ ràng, dễ hiểu.**     
            3. **Chỉ sử dụng thông tin từ ngữ cảnh được cung cấp. ** Tuyệt đối **không bịa đặt** thông tin. Nếu không thể trả lời dựa trên ngữ cảnh, hãy nói: "Xin lỗi, tôi không thể trả lời câu hỏi này của bạn."
            4. **Câu trả lời được dùng từ phần nào của ngữ cảnh thì phải trích dẫn nguồn**, quy tắc trích dẫn là nguồn bao gồm tên văn bản pháp luật (ví dụ như luật, nghị định, thông tư,...), chương, mục và điều khoản.  
                Ví dụ trích dẫn nguồn:  
                    - *Luật Hôn nhân và Gia đình, Chương II, Điều 8.*  
                    - *Bộ luật Lao động, Mục 3, Điều 34.*"""
        
        chat_history.extend([
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": query + " RESPONSE UNDER 150 TOKENS"}
        ])  

        payload = {
            "model": 'gemma3:27b',
            "messages": chat_history,
            "stream": False,
            "keep_alive": 0,
            "options": {
                "num_ctx": 3072,
                "temperature": 0.2,
                "top_k": 64,
                "top_p": 0.9,
                }
            }
        
        response = query_ollama(payload=payload)

    chat_history.append({"role": "assistant", "content": response})  

    return chat_history

def reset_chat():
    global chat_history
    # query_decomposition.to('cuda')
    chat_history = []
    return []

with gr.Blocks(theme="soft") as demo:
    gr.Markdown("<h2 style='text-align: center;'>Trợ lý Pháp luật Việt Nam 🇻🇳</h2>", elem_id="title")

    with gr.Column():
        chatbot_ui = gr.Chatbot(label="Trợ lý Pháp luật", type="messages", elem_id="chatbot", height=700)

    with gr.Row(elem_id="bottom_section"):
        query_input = gr.Textbox(placeholder="Nhập câu hỏi của bạn...", label="Câu hỏi", elem_id="input_box")

    with gr.Row():
        submit_btn = gr.Button(" ↗️ Gửi", elem_id="send_btn")
        reset_btn = gr.Button("  🔄 Đoạn chat mới", elem_id="reset_btn")

    submit_btn.click(chatbot, inputs=query_input, outputs=chatbot_ui)
    reset_btn.click(reset_chat, outputs=chatbot_ui)

    gr.Markdown("<p style='text-align: center; font-size: 12px; color: gray;'>🚀 Được phát triển bởi nhóm OPENAPI để hỗ trợ pháp luật</p>")

demo.launch(share=True)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://16c0fdde70ecea2629.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
